In [7]:
from tabula import read_pdf
from itertools import *
import os
import pandas as pd
import numpy as np
from common import *

In [18]:
IN_PDF_FILENAME = 'asd.pdf'
OUT_CSV_FILENAME = 'result.csv'

In [19]:
dfs = read_pdf(IN_PDF_FILENAME, multiple_tables=True, pages='all')

Got stderr: abr 04, 2020 4:28:05 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFORMACIÓN: OpenType Layout tables used in font ABCDEE+Calibri,BoldItalic are not implemented in PDFBox and will be ignored
abr 04, 2020 4:28:05 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFORMACIÓN: OpenType Layout tables used in font ABCDEE+Calibri,Bold are not implemented in PDFBox and will be ignored
abr 04, 2020 4:28:05 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFORMACIÓN: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
abr 04, 2020 4:28:05 PM org.apache.pdfbox.rendering.PDFRenderer suggestKCMS
INFORMACIÓN: Your current java version is: 1.8.0_181
abr 04, 2020 4:28:05 PM org.apache.pdfbox.rendering.PDFRenderer suggestKCMS
INFORMACIÓN: To get higher rendering speed on old java 1.8 or 9 versions,
abr 04, 2020 4:28:05 PM org.apache.pdfbox.rendering.PDFRenderer suggestKCMS
INFORMACIÓN:   update to the latest 1.8 or 9

In [20]:
COLUMNAS = ['Departamento/ Localidad','Confirmados','Descartados','Sospechosos','Total']

def fill_department(place_name):
    is_dep = any([x.islower() for x in place_name])
    place_name = normalize_str(place_name)
    return ('D_' if is_dep else '') + place_name
    
def normal_proc(df):
    df = dfs[0].fillna(0)
    df = df.rename(columns = {'Confirmado': 'Confirmados',
                             'Descartado': 'Descartados',
                             'Sospechoso': 'Sospechosos'
                            })
    if df['Total'].sum()==0 and 'Unnamed: 0' in df.columns:
        df['Total'] = df['Unnamed: 0']
        df = df.drop(columns='Unnamed: 0')    
    assert set(df.columns) == set(COLUMNAS)
    df['Departamento/ Localidad']=df['Departamento/ Localidad'].apply(fill_department)
    return df
def default_colval(col):
   return ( 0 if col.startswith('Unnamed') else int(float(col)) )
def second_pages(df):
    df = df.fillna(0)
    to_delete = [col for col in df.columns if col.startswith('Unnamed') and len(df[df[col]==0])>=(len(df)*0.95)]
    for c in to_delete:
        for _,place_warn in df[df[c]>0][df.columns[0]].iteritems():
            print('Warning at place: {}'.format(place_warn))
    df = df.drop(columns=to_delete)
    assert len(df.columns)==5
    df_head = pd.DataFrame(data = [ [df.columns[0]] + [ default_colval(c) for c in df.columns[1:] ] ], columns=COLUMNAS)
    df = df.rename(columns = { df.columns[i]: COLUMNAS[i] for i in range(5) })
    df = pd.concat([df_head,df])
    return df

In [21]:
df_base = normal_proc(dfs[0])
for i in range(1,len(dfs)-1):
    print(i)
    df = second_pages(dfs[i])
    df = normal_proc(df)
    df_base = pd.concat([df_base, df])
df_base.to_csv(OUT_CSV_FILENAME)

1
2
